<a href="https://colab.research.google.com/github/Quantiac/Crypto-Screener.ipynb/blob/main/Crypto-Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, time, operator, tabulate, IPython, json
import numpy as np

In [ ]:
# how many tokens to include in the final dataset:
HOW_MANY = 50
# wait time between requests to the API; increase to avoid Error 429:
API_WAIT = 3 # seconds
# timeout limit for API request:
API_TIMEOUT = 20 # seconds

CGK_API_URL = "https://api.coingecko.com/api/v3"
CGK_MARKET_FILTER = "okex"

CGK_API_ENDPOINT_TOKENS_PREFIX = "/coins/markets"
CGK_API_ENDPOINT_TOKENS_PARAMS = {
    "vs_currency": "usd",
    "order": "market_cap_asc",
    "per_page": "250",
}
CGK_API_ENDPOINT_TOKENS_FEATURES = ["/id", "/market_cap", "/name", "/symbol"]

CGK_API_ENDPOINT_TICKERS_PREFIX = "/coins"
CGK_API_ENDPOINT_TICKERS_SUFFIX = "/tickers"
CGK_API_ENDPOINT_TICKERS_PARAMS = {
    "exchange_ids": CGK_MARKET_FILTER,
}

CGK_API_ENDPOINT_MARKETCHART_PREFIX = "/coins"
CGK_API_ENDPOINT_MARKETCHART_SUFFIX = "/market_chart"
CGK_API_ENDPOINT_MARKETCHART_PARAMS = {
    "vs_currency": "usd", 
    "days": "6",
    "interval": "daily",
}
CGK_API_ENDPOINT_MARKETCHART_FEATURES = ["/market_caps", "/total_volumes"]

In [ ]:
def makeAPIRequest(base_url, endpoint_prefix, params_dict, endpoint_insert="", endpoint_suffix=""):
  # construct and make API request
  time.sleep(API_WAIT)
  request_url = base_url + endpoint_prefix + endpoint_insert + endpoint_suffix
  with requests.get(request_url, params=params_dict, timeout=API_TIMEOUT) as response:
    if response.status_code != 200:
      raise Exception("Coingecko API error. Status code: " + str(response.status_code))
    else:
      response_json = response.json()
      return response_json
  

In [ ]:
def filterFeatures(data_array, paths_dict):
  # filters JSON data to keep only selected keys
  data_filtered_features = []
  for entry in data_array:
    entry_features = {}
    for path in paths_dict:
      feature = entry
      path = [p for p in path.split("/") if p != ""]
      for path_element in path:
        feature = feature[path_element]
      entry_features[path_element] = feature
    data_filtered_features.append(entry_features)
  return data_filtered_features

In [ ]:
def getSortedTokens(page=1):
  # getting tokens tracked by CGK, sorted by market cap
  # (order is configurable in CGK_API_ENDPOINT_TOKENS_PARAMS)
  CGK_API_ENDPOINT_TOKENS_PARAMS["page"] = page
  try:
    sorted_tokens = makeAPIRequest(base_url=CGK_API_URL, 
                                  endpoint_prefix=CGK_API_ENDPOINT_TOKENS_PREFIX, 
                                  params_dict=CGK_API_ENDPOINT_TOKENS_PARAMS)
  except Exception as e:
    print("(error) ", e)
  sorted_tokens = filterFeatures(sorted_tokens, CGK_API_ENDPOINT_TOKENS_FEATURES)
  # removing tokens with 0 market cap (i.e with unknown circulating supply)
  sorted_tokens = [token for token in sorted_tokens if token["market_cap"] != 0]
  return sorted_tokens

In [ ]:
def checkOKXListing(token):
  # checking if token is listed on OKX
  try: 
    response = makeAPIRequest(base_url=CGK_API_URL, 
                  endpoint_prefix=CGK_API_ENDPOINT_TICKERS_PREFIX,
                  endpoint_insert="/" + token["id"], 
                  endpoint_suffix=CGK_API_ENDPOINT_TICKERS_SUFFIX,
                  params_dict=CGK_API_ENDPOINT_TICKERS_PARAMS)
  except Exception as e:
    print("(error)", e)
  if len(response["tickers"]) != 0:
    return True
  else:
    return False

In [ ]:
def getTurnoverRate(token_id):
  # retrieves trading vol and market cap data over the past 7 days
  # turnover rate = avg. trading vol / avg. market cap
  response = makeAPIRequest(base_url=CGK_API_URL,
                        endpoint_prefix=CGK_API_ENDPOINT_MARKETCHART_PREFIX,
                        endpoint_insert="/" + token_id,
                        endpoint_suffix=CGK_API_ENDPOINT_MARKETCHART_SUFFIX,
                        params_dict=CGK_API_ENDPOINT_MARKETCHART_PARAMS)
  filtered_data = filterFeatures([response], CGK_API_ENDPOINT_MARKETCHART_FEATURES)[0]
  volumes = [i[1] for i in filtered_data["total_volumes"]]
  market_caps = [i[1] for i in filtered_data["market_caps"]]
  turnover_rate = np.average(volumes) / np.average(market_caps)
  return turnover_rate

In [ ]:
def displayProgress(page, total, listing_check_iteration=False, listing_check_total=False, done=False):
  # custom function to display progress updates
  if done:
    print("page: {}.".format(page))
    print("found {} tokens total. done.".format(total))
    return
  print("finding tokens with low (but known) market cap listed on okx:")
  print("page: {}...".format(page))
  if listing_check_iteration:
    print("checking if token {} out of {} is listed on okx...".format(listing_check_iteration, listing_check_total))  
  print("found {} tokens total...".format(total))
  IPython.display.clear_output(True)

In [ ]:
# in case the API times out, initialising the following parameters in a separate 
# cell makes it possible to just resume, instead of starting over.
done = False
page = 1
listed_tokens = [] 

In [ ]:
while not done:
  displayProgress(page=page, total=len(listed_tokens))
  sorted_tokens = getSortedTokens(page)
  i = 0
  for token in sorted_tokens:
    displayProgress(page=page, 
             total=len(listed_tokens), 
             listing_check_iteration=i, 
             listing_check_total=len(sorted_tokens))
    isListedOKX = checkOKXListing(token)
    if isListedOKX:
      listed_tokens.append(token)
      if len(listed_tokens) >= HOW_MANY:
        displayProgress(page=page, total=len(listed_tokens), done=True)
        done = True
        break      
    i += 1
  page += 1

In [ ]:
# writing preliminary results to file, so they can simply be loaded 
# in the next step
f = open("listed_tokens.json", "w")
js = json.dumps(listed_tokens)
f.write(js)
f.close()
print("done.")

In [ ]:
f = open("listed_tokens.json", "r")
listed_tokens = json.load(f)
f.close()

tokens_with_turnover_rate = []
i = 0
for entry in listed_tokens:
  i += 1

  print("calculating turnover rates...")
  print("token {} out of {}...".format(i, len(listed_tokens)))
  IPython.display.clear_output(True)

  cgk_id = entry["id"]
  turnover_rate = getTurnoverRate(cgk_id)
  entry["turnover_rate"] = turnover_rate
  tokens_with_turnover_rate.append(entry)
tokens_with_turnover_rate.sort(key=operator.itemgetter("turnover_rate"), reverse=True)
print("done.")

In [ ]:
formatted_result = [[token["name"],
                     token["turnover_rate"],
                     "{:,}".format(token["market_cap"]),
                     token["id"],
                     ] for token in tokens_with_turnover_rate]
tabulated = tabulate.tabulate(formatted_result, headers = ["Name", "Turnover Rate", "Market Cap (USD)", "Coingecko ID"], stralign="right")
header = "# tokens with lowest known market cap, that are listed on okx, sorted by turnover rate\n\n"
print(header)
print(tabulated)
f = open("result.txt", "w")
f.write(header)
f.write(tabulated)
f.close()

# tokens with lowest known market cap, that are listed on okx, sorted by turnover rate


                 Name    Turnover Rate    Market Cap (USD)           Coingecko ID
---------------------  ---------------  ------------------  ---------------------
               Primas       1.79071                441,722                 primas
            TrueChain       0.836723               371,674             true-chain
                   GM       0.62827              3,343,648                     gm
       Pickle Finance       0.612595             1,234,532         pickle-finance
              RioDeFi       0.545436             1,742,687               rio-defi
           Value DeFi       0.515278             1,252,834        value-liquidity
             FairGame       0.477083               812,616               fairgame
Trabzonspor Fan Token       0.408446             4,067,417  trabzonspor-fan-token
  Blockchain Brawlers       0.389721             5,064,114    blockchain-brawlers
       Co